# 2-1 extract 4grams from smali code

- step1 smali 코드 수집
- step2 smali 코드에서 4gram을 추출하여 전체 리스트 및 갯수 확인
- step3 smali 코드에서 비번하게 나타나는 10000개의 리스트를 가지고 malware, benign의 비율 확인
- step4 10000개의 리스트 중에서 영향도 있는 리스트만 각출
- step5 각 apk 파일들의 4gram 수집 및 정리


## step1 smali 코드 수집

In [4]:
import os

dir_names = ["Benign_samples_1st_1500", "Malware_samples_1st_500"]
dir_list = dict()

i = 0
print( ".",end =' ')
for dir_name in dir_names:
    dir_name = dir_name + "_apktools"
    if dir_name.startswith('Benign'):
        tag = 'Benign'
    else :
        tag = 'Malware'
        
    for root, dirs, files in os.walk(dir_name):
        for f in files:
            if f.endswith(".smali"):
                dir_list[root + '/' + f ] = tag
            i+=1
            if i%1000 == 0:
                print( i, end =' ')
                    
print('len', len(dir_list))

. 1000 2000 3000 4000 5000 6000 7000 8000 9000 10000 11000 12000 13000 14000 15000 16000 17000 18000 19000 20000 21000 22000 23000 24000 25000 26000 27000 28000 29000 30000 31000 32000 33000 34000 35000 36000 37000 38000 39000 40000 41000 42000 43000 44000 45000 46000 47000 48000 49000 50000 51000 52000 53000 54000 55000 56000 57000 58000 59000 60000 61000 62000 63000 64000 65000 66000 67000 68000 69000 70000 71000 72000 73000 74000 75000 76000 77000 78000 79000 80000 81000 82000 83000 84000 85000 86000 87000 88000 89000 90000 91000 92000 93000 94000 95000 96000 97000 98000 99000 100000 101000 102000 103000 104000 105000 106000 107000 108000 109000 110000 111000 112000 113000 114000 115000 116000 117000 118000 119000 120000 121000 122000 123000 124000 125000 126000 127000 128000 129000 130000 131000 132000 133000 134000 135000 136000 137000 138000 139000 140000 141000 142000 143000 144000 145000 146000 147000 148000 149000 150000 151000 152000 153000 154000 155000 156000 157000 158000 

In [2]:
#dir_list.keys()[:10]
a = dir_list.keys()
a = list(a)
a[:10]

['Benign_samples_1st_1500_apktools/b54a9bd9669e2bca118a58c4e08bafad9bacfa5717364226140b9d9180a2b5e0/smali/com/appsbar/CBFerreterias140547/ActivityWMenu$FbLoginDialogListener.smali',
 'Benign_samples_1st_1500_apktools/b54a9bd9669e2bca118a58c4e08bafad9bacfa5717364226140b9d9180a2b5e0/smali/com/appsbar/CBFerreterias140547/LauncherActivity.smali',
 'Benign_samples_1st_1500_apktools/b54a9bd9669e2bca118a58c4e08bafad9bacfa5717364226140b9d9180a2b5e0/smali/com/appsbar/CBFerreterias140547/ActivityWMenu$WallPostListener.smali',
 'Benign_samples_1st_1500_apktools/b54a9bd9669e2bca118a58c4e08bafad9bacfa5717364226140b9d9180a2b5e0/smali/com/appsbar/CBFerreterias140547/ActivityWMenu$1.smali',
 'Benign_samples_1st_1500_apktools/b54a9bd9669e2bca118a58c4e08bafad9bacfa5717364226140b9d9180a2b5e0/smali/com/appsbar/CBFerreterias140547/R$style.smali',
 'Benign_samples_1st_1500_apktools/b54a9bd9669e2bca118a58c4e08bafad9bacfa5717364226140b9d9180a2b5e0/smali/com/appsbar/CBFerreterias140547/LauncherActivity$RunAppL

## step2 smali 코드에서 4gram을 추출하여 전체 리스트 및 갯수 확인

In [3]:
import nltk
from multiprocessing import Pool
from collections import Counter
import numpy as np

# Create three Counter objects to store positive, negative and total counts
global benign_list
global malware_list
global total_counts

benign_list = Counter()
malware_list = Counter()
total_counts = Counter()

def file2strings(key):
    special_char = ['$',',','.',';','-','=','<','>',
                    '1','2','3','4','5','6','7','8','9','0',
                    '{','}','(',')',':','`', '#', "'",'/','"']
    
    with open(key) as f:
        data = f.read()#.encode('ascii', 'replace')
        #data = data.decode('utf-8', 'ignore')
        data = data.lower()
        
        for char in special_char:
            data = data.replace(char,' ')
        
        data = data.split()
        fourgrams = nltk.ngrams(data, 4)
        
        return dir_list[key], list(fourgrams)

In [8]:
with Pool(16) as p:
    count = len(dir_list)
    print ("start")
    i = 0
    for key, data in p.imap(file2strings, list(dir_list.keys())):
            if key == 'Benign':
                for word in data:
                    total_counts[word] += 1
                    benign_list[word] += 1

            else :
                for word in data:
                    total_counts[word] += 1
                    malware_list[word] += 1
                    
            i += 1    
            if i%100 == 0:
                print("{}%                              \r".format(i/count*100),flush=True,sep=' ', end = '')
    print("{}%                              \r".format(i/count*100),flush=True,sep=' ', end = '')

start


In [9]:
len(total_counts)

18743295

In [50]:
topCut = total_counts.most_common()[:100000]

In [51]:
topCut[-1]

(('lcom', 'applovin', 'impl', 'bootstrap'), 685)

## step3 smali 코드에서빈번하게 나타나는 10000개의 리스트를 가지고 malware, benign의 비율 확인

In [13]:
benign_list.most_common()[:100]

[(('move', 'result', 'object', 'v'), 8811641),
 (('iget', 'object', 'v', 'p'), 7485551),
 (('object', 'v', 'p', 'lcom'), 6808035),
 (('invoke', 'virtual', 'v', 'v'), 5746842),
 (('lcom', 'google', 'ads', 'ag'), 4508178),
 (('p', 'lcom', 'appsbar', 'cbferreterias'), 3668360),
 (('lcom', 'appsbar', 'cbferreterias', 'launcheractivity'), 3582185),
 (('v', 'invoke', 'virtual', 'v'), 3532313),
 (('ljava', 'lang', 'string', 'ljava'), 3515571),
 (('result', 'object', 'v', 'invoke'), 3490722),
 (('lang', 'string', 'ljava', 'lang'), 3409290),
 (('line', 'iget', 'object', 'v'), 3238010),
 (('v', 'ljava', 'lang', 'stringbuilder'), 3174761),
 (('v', 'p', 'lcom', 'appsbar'), 3063586),
 (('v', 'v', 'ljava', 'lang'), 2919428),
 (('lcom', 'appsbar', 'cbferreterias', 'utilities'), 2880168),
 (('end', 'method', 'method', 'public'), 2819218),
 (('virtual', 'v', 'v', 'ljava'), 2575246),
 (('object', 'v', 'invoke', 'virtual'), 2452535),
 (('v', 'p', 'lcom', 'google'), 2424489),
 (('v', 'lcom', 'appsbar', 'c

In [14]:
malware_list.most_common()[:100]

[(('move', 'result', 'object', 'v'), 2611824),
 (('iget', 'object', 'v', 'p'), 1694659),
 (('invoke', 'virtual', 'v', 'v'), 1679888),
 (('object', 'v', 'p', 'lcom'), 1424739),
 (('v', 'invoke', 'virtual', 'v'), 1205701),
 (('ljava', 'lang', 'string', 'ljava'), 1168704),
 (('lang', 'string', 'ljava', 'lang'), 1145721),
 (('result', 'object', 'v', 'invoke'), 1131884),
 (('v', 'ljava', 'lang', 'stringbuilder'), 1103789),
 (('v', 'v', 'ljava', 'lang'), 960618),
 (('ljava', 'lang', 'string', 'move'), 850954),
 (('lang', 'string', 'move', 'result'), 844513),
 (('string', 'move', 'result', 'object'), 844507),
 (('virtual', 'v', 'v', 'ljava'), 760224),
 (('object', 'v', 'invoke', 'virtual'), 727204),
 (('if', 'eqz', 'v', 'cond_'), 647716),
 (('end', 'method', 'method', 'public'), 643161),
 (('v', 'const', 'v', 'x'), 622929),
 (('return', 'void', 'end', 'method'), 587707),
 (('invoke', 'virtual', 'v', 'ljava'), 583612),
 (('string', 'ljava', 'lang', 'string'), 580640),
 (('ljava', 'lang', 'stri

In [52]:
benign_malware_ratios = Counter()

# Calculate the ratios of positive and negative uses of the most common words
# Consider words to be "common" if they've been used at least 100 times
for term, cnt in list(topCut):
    benign_malware_ratio = benign_list[term]+3 / (float(malware_list[term]+1)*3)
    benign_malware_ratios[term] = benign_malware_ratio

for word,ratio in list(benign_malware_ratios.most_common()):
    benign_malware_ratios[word] = np.log(ratio)

word1 = ('stringbuilder', 'move', 'result', 'object')
word2 = ('field', 'public', 'static', 'final')
word3 = ('lcom', 'google', 'ads', 'ag')
print("benign_to_malware ratio for '{}' = {}".format(word1, benign_malware_ratios[word1]))
print("benign_to_malware ratio for '{}' = {}".format(word2, benign_malware_ratios[word2]))
print("benign_to_malware ratio for '{}' = {}".format(word3, benign_malware_ratios[word3]))

benign_to_malware ratio for '('stringbuilder', 'move', 'result', 'object')' = 14.214075917459589
benign_to_malware ratio for '('field', 'public', 'static', 'final')' = 14.302301242827811
benign_to_malware ratio for '('lcom', 'google', 'ads', 'ag')' = 15.321403638723847


In [53]:
# words most frequently seen in a review with a "POSITIVE" label
list(benign_malware_ratios.most_common())[:100]

[(('move', 'result', 'object', 'v'), 15.991584246174385),
 (('iget', 'object', 'v', 'p'), 15.82848518702103),
 (('object', 'v', 'p', 'lcom'), 15.733614090232811),
 (('invoke', 'virtual', 'v', 'v'), 15.564161044507216),
 (('lcom', 'google', 'ads', 'ag'), 15.321403638723847),
 (('p', 'lcom', 'appsbar', 'cbferreterias'), 15.115255526272501),
 (('lcom', 'appsbar', 'cbferreterias', 'launcheractivity'),
  15.091483786552828),
 (('v', 'invoke', 'virtual', 'v'), 15.077463455126395),
 (('ljava', 'lang', 'string', 'ljava'), 15.072712516691301),
 (('result', 'object', 'v', 'invoke'), 15.065619149573642),
 (('lang', 'string', 'ljava', 'lang'), 15.042014616437312),
 (('line', 'iget', 'object', 'v'), 14.990469501541886),
 (('v', 'ljava', 'lang', 'stringbuilder'), 14.970742911885313),
 (('v', 'p', 'lcom', 'appsbar'), 14.935097009634763),
 (('v', 'v', 'ljava', 'lang'), 14.886898264644829),
 (('lcom', 'appsbar', 'cbferreterias', 'utilities'), 14.873359530946047),
 (('end', 'method', 'method', 'public')

In [54]:
# words most frequently seen in a review with a "NEGATIVE" label
list(reversed(benign_malware_ratios.most_common()))[:100]

[(('p', 'lcom', 'dd', 'launcher'), -13.052657394571053),
 (('v', 'p', 'lcom', 'dd'), -12.858168638640505),
 (('v', 'lcom', 'dd', 'launcher'), -12.635107785150771),
 (('lcom', 'dd', 'launcher', 'theme'), -12.301137340660146),
 (('lcom', 'madhouse', 'android', 'ads'), -12.049646879656187),
 (('v', 'v', 'lcom', 'dd'), -11.800657536172052),
 (('v', 'p', 'lcom', 'waps'), -11.798968695253484),
 (('lnet', 'youmi', 'android', 'appoffers'), -11.709667544425413),
 (('lcom', 'dd', 'launcher', 'launcher'), -11.696671007415201),
 (('lcom', 'adchina', 'android', 'ads'), -11.666170020104762),
 (('a', 'lcom', 'dd', 'launcher'), -11.577823296548072),
 (('p', 'lcom', 'guiji', 'launcher'), -11.499586900531048),
 (('lcom', 'apperhand', 'common', 'dto'), -11.335185397109868),
 (('v', 'lcom', 'airpush', 'android'), -11.277165156044557),
 (('v', 'p', 'lcom', 'guiji'), -11.273106142645405),
 (('lcom', 'google', 'mygson', 'stream'), -11.227840018365713),
 (('v', 'lcom', 'guiji', 'launcher'), -11.09262586790360

In [61]:
rank = 7000
print((benign_malware_ratios.most_common())[rank])
print(list(reversed(benign_malware_ratios.most_common()))[rank])

(('ioexception', 'lcom', 'flurry', 'org'), 9.3467052464966986)
(('\\u', 'b', '\\u', 'fb\\u'), 3.9512588014239403)


## 영향도 있는 benign_malware_ratios 리스트 저장

benign과 malware에 있는 영향도 있는 benign_malware_ratios를 저장한다.

In [28]:
import pickle

with open('benign_malware_ratios.pkl', 'wb') as f:
    pickle.dump(benign_malware_ratios, f)

In [1]:
import pickle

with open('benign_malware_ratios.pkl', 'rb') as f:
    benign_malware_ratios = pickle.load(f)

from collections import Counter 
benign_malware_ratios.most_common()

[(('move', 'result', 'object', 'v'), 15.991584246174385),
 (('iget', 'object', 'v', 'p'), 15.82848518702103),
 (('object', 'v', 'p', 'lcom'), 15.733614090232811),
 (('invoke', 'virtual', 'v', 'v'), 15.564161044507216),
 (('lcom', 'google', 'ads', 'ag'), 15.321403638723847),
 (('p', 'lcom', 'appsbar', 'cbferreterias'), 15.115255526272501),
 (('v', 'invoke', 'virtual', 'v'), 15.077463455126395),
 (('ljava', 'lang', 'string', 'ljava'), 15.072712516691301),
 (('result', 'object', 'v', 'invoke'), 15.065619149573642),
 (('lang', 'string', 'ljava', 'lang'), 15.042014616437312),
 (('line', 'iget', 'object', 'v'), 14.990469501541886),
 (('v', 'ljava', 'lang', 'stringbuilder'), 14.970742911885313),
 (('v', 'p', 'lcom', 'appsbar'), 14.935097009634763),
 (('v', 'v', 'ljava', 'lang'), 14.886898264644829),
 (('lcom', 'appsbar', 'cbferreterias', 'utilities'), 14.873359530946047),
 (('end', 'method', 'method', 'public'), 14.851970099494178),
 (('virtual', 'v', 'v', 'ljava'), 14.761455621497987),
 (('o

In [6]:
import pandas as pd
limit = 7000
df = pd.DataFrame(list(benign_malware_ratios.most_common())[:limit] + list(benign_malware_ratios.most_common())[-limit:])
df

,0,1
0,"(move, result, object, v)",15.991584
1,"(iget, object, v, p)",15.828485
2,"(object, v, p, lcom)",15.733614
3,"(invoke, virtual, v, v)",15.564161
4,"(lcom, google, ads, ag)",15.321404
5,"(p, lcom, appsbar, cbferreterias)",15.115256
6,"(v, invoke, virtual, v)",15.077463
7,"(ljava, lang, string, ljava)",15.072713
8,"(result, object, v, invoke)",15.065619
9,"(lang, string, ljava, lang)",15.042015


In [7]:
word_list = list(df[0])
word_list[:100]

[('move', 'result', 'object', 'v'),
 ('iget', 'object', 'v', 'p'),
 ('object', 'v', 'p', 'lcom'),
 ('invoke', 'virtual', 'v', 'v'),
 ('lcom', 'google', 'ads', 'ag'),
 ('p', 'lcom', 'appsbar', 'cbferreterias'),
 ('v', 'invoke', 'virtual', 'v'),
 ('ljava', 'lang', 'string', 'ljava'),
 ('result', 'object', 'v', 'invoke'),
 ('lang', 'string', 'ljava', 'lang'),
 ('line', 'iget', 'object', 'v'),
 ('v', 'ljava', 'lang', 'stringbuilder'),
 ('v', 'p', 'lcom', 'appsbar'),
 ('v', 'v', 'ljava', 'lang'),
 ('lcom', 'appsbar', 'cbferreterias', 'utilities'),
 ('end', 'method', 'method', 'public'),
 ('virtual', 'v', 'v', 'ljava'),
 ('object', 'v', 'invoke', 'virtual'),
 ('v', 'p', 'lcom', 'google'),
 ('ljava', 'lang', 'string', 'move'),
 ('string', 'move', 'result', 'object'),
 ('lang', 'string', 'move', 'result'),
 ('return', 'void', 'end', 'method'),
 ('v', 'lcom', 'google', 'ads'),
 ('v', 'v', 'lcom', 'google'),
 ('p', 'lcom', 'google', 'ads'),
 ('if', 'eqz', 'v', 'cond_'),
 ('v', 'line', 'local', '

## step5 각 apk 파일들의 4gram 수집 및 정리

apk_word_list

파일로부터 데이터를 불러와서 해시 값에 따라 counter 값 추가

In [9]:
import nltk
from multiprocessing import Pool
from collections import Counter
import numpy as np

apk_word_list = dict()

def file2strings(key):
    special_char = ['$',',','.',';','-','=','<','>',
                    '1','2','3','4','5','6','7','8','9','0',
                    '{','}','(',')',':','`', '#', "'",'/','"']
    
    with open(key) as f:
        data = f.read()#.encode('ascii', 'replace')
        #data = data.decode('utf-8', 'ignore')
        data = data.lower()
        
        for char in special_char:
            data = data.replace(char,' ')
        
        data = data.split()
        fourgrams = nltk.ngrams(data, 4)
                
        word_count = Counter()
        filename = key.split("/")[1] # 'Benign_samples_1st_1500_apktools/b54a9bd9669e2bca118a58c4e08bafad9bacfa5717364226140b9d9180a2b5e0/smali/com/appsbar/CBFerreterias140547/ActivityWMenu$FbLoginDialogListener.smali'
        
        for word in list(fourgrams):
            if word in word_list:
                word_count[str(word)] += 1
                
        return filename, dict(word_count.most_common())

with Pool(32) as p:
    print ("start")
    i = 0
    count = len(dir_list)
    
    for filename, word_count in p.imap(file2strings, list(dir_list.keys())):
        apk_word_list[filename] = word_count
        
        i += 1    
        if i %100 == 0:
            print("{}%                              \r".format(i/count*100),flush=True,sep=' ', end = '')
     
    print("{}%                              \r".format(i/count*100),flush=True,sep=' ', end = '')

start


In [14]:
apk_word_df = pd.DataFrame(apk_word_list)
apk_word_df = apk_word_df.fillna(0).T
apk_word_df

,"('&', 'invoke', 'virtual', 'v')","('*', 'end', 'annotation', 'end')","('*', 'end', 'annotation', 'prologue')","('*', 'iget', 'object', 'v')","('[', 'invoke', 'virtual', 'v')","('[b', 'array', 'length', 'v')","('[b', 'const', 'v', 'x')","('[b', 'end', 'local', 'v')","('[b', 'fill', 'array', 'data')","('[b', 'i', 'move', 'result')",...,"('z', 'v', 'line', 'cond_')","('z', 'v', 'line', 'const')","('z', 'v', 'line', 'iget')","('z', 'v', 'line', 'invoke')","('z', 'v', 'line', 'return')","('z', 'v', 'locals', 'param')","('z', 'v', 'locals', 'prologue')","('z', 'v', 'return', 'void')","('z', 'v', 'try_end_', 'catch')","('z', 'z', 'move', 'result')"
000b14d01e2416f3c4b8d260a20021ec477f543993e1f4041f0efd7c95f00bfb,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
002f4f25130c0090b7b8ece3e3b41a3e082e224f77159e665d67e3d64a358c85,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
004679da462ba3ca1fff77c8d8e1f5a057f2eaf30f0feb3f07320992619ff9d1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
0081079f409a40cbb81c5a3b47dd56242bcda516aeb79ec13a13b0e913162d39,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
00b04c9a5966b0d64c135ee3894a9410ecafb097a6b342619274b16d7fea4c4e,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
00b28b5428689c82cda142420f220c2be821eaf913fc951187a7099f39e6fa0e,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,2.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
00b838179a351a6cfd77af944c35a1758dfaa0a6ddba04fc423f3d6409e50ee4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
00cb6d24d37ee40bef1c9ac231880b50bcf45c554ef863ed4f16554de6720885,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
00ea77eb6f076fe87adb3cd9d39bcb3e8e487529faed7c8853c5a821be801e0a,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
012432a7b76a23d68d18fbcd88a2cf13c0c89e94351e55f65582c18738197a82,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [15]:
apk_word_df.describe()

,"('&', 'invoke', 'virtual', 'v')","('*', 'end', 'annotation', 'end')","('*', 'end', 'annotation', 'prologue')","('*', 'iget', 'object', 'v')","('[', 'invoke', 'virtual', 'v')","('[b', 'array', 'length', 'v')","('[b', 'const', 'v', 'x')","('[b', 'end', 'local', 'v')","('[b', 'fill', 'array', 'data')","('[b', 'i', 'move', 'result')",...,"('z', 'v', 'line', 'cond_')","('z', 'v', 'line', 'const')","('z', 'v', 'line', 'iget')","('z', 'v', 'line', 'invoke')","('z', 'v', 'line', 'return')","('z', 'v', 'locals', 'param')","('z', 'v', 'locals', 'prologue')","('z', 'v', 'return', 'void')","('z', 'v', 'try_end_', 'catch')","('z', 'z', 'move', 'result')"
count,1994.000000,1994.000000,1994.000000,1994.000000,1994.000000,1994.00000,1994.000000,1994.000000,1994.000000,1994.000000,...,1994.000000,1994.000000,1994.000000,1994.000000,1994.000000,1994.000000,1994.000000,1994.000000,1994.000000,1994.000000
mean,0.006520,0.001505,0.005015,0.002006,0.007523,0.00652,0.055165,0.013039,0.013039,0.027081,...,0.006018,0.014042,0.093280,0.120361,0.011535,0.036108,0.047141,0.010532,0.007021,0.040622
std,0.269652,0.050065,0.070657,0.054832,0.097349,0.08050,0.568416,0.223619,0.144581,0.188129,...,0.118376,0.147919,0.503201,0.786589,0.124183,0.250839,0.387699,0.143042,0.083518,0.259017
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
max,12.000000,2.000000,1.000000,2.000000,2.000000,1.00000,15.000000,8.000000,3.000000,4.000000,...,4.000000,2.000000,5.000000,6.000000,3.000000,4.000000,6.000000,4.000000,1.000000,5.000000


In [16]:
len(list(apk_word_df))

7361

In [17]:
apk_word_df.to_csv("1st_smali_4gram.csv")